In [0]:
!pip install -q xlrd  # biblioteca para extraer y leer los datos de excel
!pip install pycm # biblioteca para el analisis estadistico de matrices de confusión
!git clone https://github.com/github466/DL.git #clonar el repositorio con los archivos de la BD 

In [0]:
import csv #libreria para lectura de archivos en formato csv 
import itertools #libreria que convierte los datos a iterables de forma mas rapida 
from tensorflow.examples.tutorials.mnist import input_data #importar función imput data de los ejemplos de mnist de tensorflow
import numpy as np #biblioteca para trabajar con vectores y matrices 
import tensorflow as tf #entorno de trabajo de codigo abierto para el dieño, construcción y desarrollo de modelos de DeepLearning
import matplotlib.pyplot as plt#biblioteca para la generación de graficos a partir de datos contenidos en listas o arrays
import pylab #Api similar a la de generación de graficos de Matlab
import os #El módulo os nos permite acceder a funcionalidades dependientes del Sistema Operativo
from sklearn.metrics import confusion_matrix # libreria para genera la matriz de confusión
from pycm import *# biblioteca para el analisis estadistico de matrices de confusión

In [0]:
def plot_confusion_matrix(cm, classes,   #función para generar la matriz de confusión para evaluar el clasificador
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [0]:
def acurrency(_logits,_y,_data_x,_data_y):
  pred = tf.nn.softmax(_logits)  # Apply softmax to logits
  correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(_y, 1))
  # Calculate accuracy
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
  eval = accuracy.eval({X: _data_x, Y: _data_y})  
  return eval

In [0]:
def generateLayers(n_input,n_classes,n_hidden): #función para generar las capas del perceptron
    weights = {}   
    biases = {}
    hidden_before=n_input
    for i in range(len(n_hidden)):
        l=i+1        
        weights['h' + str(l)]=tf.Variable(tf.random_normal([hidden_before, n_hidden[i]]))      
        biases['b' + str(l)]=tf.Variable(tf.random_normal([n_hidden[i]]))
        hidden_before = n_hidden[i]
        
    weights['out']=tf.Variable(tf.random_normal([hidden_before,n_classes]))
    biases['out']=tf.Variable(tf.random_normal([n_classes]))
    return weights,biases

In [0]:
def metrics(_sess,_logict,x_data,y_data): #función para generar los valores de especificidad y sensibilidad
    pred=tf.argmax(_logict,1)
    y_pred = pred.eval(feed_dict={X: x_data})
    t_y_real = tf.argmax(y_data,1)
    y_real = sess.run(t_y_real)
    print(type(y_pred),type(y_real))
    cm = ConfusionMatrix(y_real.tolist(), y_pred.tolist())
    #cm.TNR Especificidad o verdadesros negativos
    #cm.TPR Sensibilidad o verdaderos positivos
    return cm,cm.TNR,cm.TPR,y_real.tolist(), y_pred.tolist()


In [0]:
def multilayer_perceptron(x_,n_hidden_,weights_,biases_,dropout_rate_):  #función para generar el perceptron multicapa
    layer_before = x_
    for i in range(len(n_hidden_)):
        l=i+1   
        layer = tf.sigmoid( tf.add(tf.matmul(layer_before, weights_['h'+ str(l)]), biases_['b'+ str(l)]) )     #función sigmoidal en todas la capas del MLP
        layer_before=layer   
    # Output fully connected layer with a neuron for each class
    #drop_out = tf.nn.dropout(layer_before, dropout_rate)  # DROP-OUT here
    #out_layer = tf.matmul(drop_out, weights['out']) + biases['out']
    out_layer = tf.matmul(layer_before, weights_['out']) + biases_['out']
    out_layer = tf.sigmoid(out_layer) #funcion de activación sigmoidal en la capa de salida 
#    sigmoide o softmax
    out_layer =  tf.nn.dropout(out_layer, dropout_rate_)  # DROP-OUT en la capa de salida 
    #dropout .5 .3 .2
    return out_layer

In [0]:
def load_data_train_test(porcentTestLength,name):#función para leer los datos de train y de test ya que estan en dos archivos separados 
    datos=[]
    with open(name) as f:
        reader = csv.reader(f)
        for row in reader:
            if(row!=','):               
                datos.append(row)
    
    
    size = len(datos)    
    testLength = round(size*porcentTestLength/100)
    np.random.shuffle(datos)

    test_x,test_y = separeData(datos[:testLength])
    train_x,train_y = separeData(datos[testLength:]) 
    return datos,train_x,train_y, test_x,test_y
def load_data(name): 
    datos=[]
    with open(name) as f:
        reader = csv.reader(f)
        for row in reader:
            if(row!=','):               
                datos.append(row)
    
    
   
    np.random.shuffle(datos)
    test_x,test_y = separeData(datos)   
    return datos,test_x,test_y


In [0]:
def separeData(datos):   #Función para separar las caracteristicas de entrada de las clases, "separación vertical"
    
    X=[]
    S=[]
    for d in datos:
        X.append(d[0:178])
        S.append(d[178:len(datos[0])])   
    
   
    X=np.array(X, np.float64)
    S=np.array(S,np.float64)
    
    return X,S

In [0]:
_,train_x_orig, train_y_orig = load_data('DL/train.csv')
_,test_x_orig, test_y_orig = load_data('DL/test.csv')
print(test_x_orig.shape) #imprimir los datos de test 
print(test_x_orig[0])

In [0]:
_,train_x_orig, train_y_orig = load_data('DL/train.csv')
_,test_x_orig, test_y_orig = load_data('DL/test.csv')


print(train_x_orig.shape)
total_examples = len(train_x_orig) # numero de ejemplos de la BD
learning_rate = 0.0001 #
training_epochs = 3000 #
batch_size =30# <=10 & <= 50 # numero de ejemplos que van a ser mostrados a la red en cada itereación 
display_step = 10 #cada cuantas iteraciónes imprime los resultados de la función de costo 
n_input = 178  #numero de caracteristicas de la capa de entrada 
n_hidden=[600,500,400,300] # numero de capas oculta y cantidad de neuronas por capa 
 n_classes = 5 # numero de clases 
dropout_rate = 0.5 #.5 .3 .2 #cantidad de dropout, tecnica para la regularización por medio del apagado de algunas neuronas que pueden estar saturadas 

# tf Graph input
X = tf.placeholder("float", [None, n_input],"X") #definición de los placeholders o espacios en el grafo para los datos de entrada 
Y = tf.placeholder("float", [None, n_classes],"Y")#definicion de los place holders o espacios en el grafo para los datos 

# Store layers weight & bias
weights,biases = generateLayers(n_input,n_classes,n_hidden)
# Construct model
logits = multilayer_perceptron(X,n_hidden,weights,biases,dropout_rate)

# Define loss and optimizer

loss_op = tf.reduce_mean(tf.losses.softmax_cross_entropy(logits=logits, onehot_labels=Y))#
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)# optimizador 
train_op = optimizer.minimize(loss_op)


# Initializing the variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    #saver = tf.train.Saver()
    sess.run(init)
    epoch_min=cost_init=accurrency_init=accurrency_current=0
    cost_min=99999999 
    costs=accurrencys=[]  
    cost_bef=-cost_min    
 
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(train_x_orig)/batch_size)            
        # Loop over all batches
        for i in range(total_batch):
            start=i*batch_size
            end=(i+1)*batch_size 
            batch_x=train_x_orig[start:end]
            batch_y=train_y_orig[start:end]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,Y: batch_y}) 
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        
        if(avg_cost < cost_min):
          cost_min=avg_cost
          epoch_min=epoch          
        if epoch % display_step == 0:
            simbol = "+" if cost_bef<avg_cost else "-"            
            print(simbol,") Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost),"accurrency={:.9f}".format(accurrency_current))           
            cost_bef=avg_cost 
        if(epoch==0):
          cost_init = avg_cost
          accurrency_init =  acurrency(logits,Y,test_x_orig,test_y_orig)
          
        costs.append(avg_cost)
        accurrencys.append(accurrency_current)
    print("Optimization Finished!")    
    print("Cost Min:","cost={:.9f}".format(cost_min)," in epoch:",'%04d' % (epoch_min+1),"cost_init={:.9f}".format(cost_init),"accurrency_init={:.9f}".format(accurrency_init));
    # Test model
    pred = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy-train:", acurrency(logits,Y,train_x_orig,train_y_orig)) 
    print("Accuracy-test:", acurrency(logits,Y,test_x_orig,test_y_orig)) 
   
    cm,TNR,TPR,y_real,y_pred = metrics(sess,logits,test_x_orig,test_y_orig)
    cnf_matrix = confusion_matrix(y_real, y_pred)   
    print("TNR Especificidad o verdadesros negativos",TNR)
    print("TPR Sensibilidad o verdaderos positivos",TPR)    
    print(cm)
    plot_confusion_matrix(cnf_matrix, classes=cm.classes,title='Confusion matrix')